NES文件格式
.NES文件为模拟用来储存NES卡带的映像。下面是一个.NES文件的结构。 

偏移 | 字节数 | 内容  
-|:-|:-   
0－3  | 4  | 字符串“NES^Z”用来识别.NES文件   
4 | 1 |  16kB ROM的数目  PRG-ROM page count 
5 | 1 | 8kB VROM的数目 CHR-ROM page count 
6 | 1 | ROM Control Byte #1 <br>D0：1＝垂直镜像，0＝水平镜像 <br> D1：1＝有电池记忆，SRAM地址$6000-$7FFF <br>  D2：1＝在$7000-$71FF有一个512字节的trainer D3：1＝4屏幕VRAM布局 <br> D4－D7：ROM Mapper的低4位  
7 | 1 | ROM Control Byte #2 <br> D0－D3：保留，必须是0 <br> D4－D7：ROM Mapper的高4位  
8－F  | 8 | 保留，必须是0  
16 - ?  | 16KxM | ROM段升序排列，如果存在trainer，它的512字节摆在ROM段之前  
? - EOF | 8KxN  | VROM段, 升序排列 


In [61]:
# parser nes file
import struct
def nes_parser(filename='Spartanx.nes', offset=0x0, size=1):
    # print('parsing nes file (' + filename + ')...')
    with open(filename, 'rb') as f:
        data = f.read()

    if size == 1:
        format = 'B'
    elif size == 2:
        format = '<H'
    elif size == 4:
        format = '<I'
    elif size == 8:
        format = '<Q'
    else:
        print('invalid size', size)

    value = struct.unpack(format, data[offset : offset + size])[0]
    # print('value=', hex(value), value)
    # return hex(value)
    return value

# nes_parser(offset=0x1)

def nes_summary(filename='Spartanx.nes'):
    nes_summary = {}
    file_flag = chr(nes_parser(filename, offset=0x0)) + chr(nes_parser(filename, offset=0x1)) + chr(nes_parser(filename, offset=0x2))
    num_16k_rom = nes_parser(filename, offset=0x4)
    num_8k_vrom = nes_parser(filename, offset=0x5)
    
    nes_summary["filename"] = filename
    nes_summary["file_flag"] = file_flag
    nes_summary["num_16k_rom"] = num_16k_rom
    nes_summary["num_8k_vrom"] = num_8k_vrom
    
    # ROM Control Byte#1
    crb1 = '{:08b}'.format(nes_parser(filename, offset=0x6))    
    # '01000001'
    # D0：1＝垂直镜像，0＝水平镜像  
    nes_summary["mirroring"] = crb1[-1]
    # D1：1＝有电池记忆，SRAM地址 6000− 7FFF
    nes_summary["battery"] = crb1[-2]
    # D2：1＝在 7000− 71FF有一个512字节的trainer D3：1＝4屏幕VRAM布局
    nes_summary["trainer"] = crb1[-3]
    # D3：1＝4屏幕VRAM布局
    nes_summary["screen"] = crb1[-4]
    # D4－D7：ROM Mapper的低4位
    nes_summary["mapper_lower_4bits"] = '{}{}{}{}'.format(crb1[-5], crb1[-6], crb1[-7], crb1[-8])
    
    # ROM Control Byte#2
    crb2 = '{:08b}'.format(nes_parser(filename, offset=0x7))
    # D4－D7：ROM Mapper的高4位
    nes_summary["mapper_upper_4bits"] = '{}{}{}{}'.format(crb2[-5], crb2[-6], crb2[-7], crb2[-8])    
    print(nes_summary)
    return(nes_summary)

nes_summary()

{'filename': 'Spartanx.nes', 'file_flag': 'NES', 'num_16k_rom': 2, 'num_8k_vrom': 1, 'mirroring': '1', 'battery': '0', 'trainer': '0', 'screen': '0', 'mapper_lower_4bits': '0000', 'mapper_upper_4bits': '0000'}


{'filename': 'Spartanx.nes',
 'file_flag': 'NES',
 'num_16k_rom': 2,
 'num_8k_vrom': 1,
 'mirroring': '1',
 'battery': '0',
 'trainer': '0',
 'screen': '0',
 'mapper_lower_4bits': '0000',
 'mapper_upper_4bits': '0000'}

In [60]:
# Game list
# 根据游戏目录128in1 中的nes 文件 自动生成游戏列表文件 game-list-<时间戳>.xls
# 可以填充 序号	版本	文件名	 程序容量(PRG)	图像容量(CHR)	镜像 字段
# 这个电子表格文件将作为自动生成 F_Reset.asm, Testnes.idx 代码内容的输入源
#

import os
import time
from openpyxl import load_workbook,Workbook

nes_files_dir = './128in1/'
nes_games_list = 'Nes-Games-List-' + str(time.time()) + '.xlsx'

print('Start create ' + nes_games_list + '...')

wb = Workbook()
ws = wb.create_sheet("Sheet1")

ws.append(['序号', '版本', '英文名',  '中文名', '文件名', '程序容量(PRG)16KxM', '图像容量(CHR)8KxN','镜像(1垂直 0水平)', '预置'])

idx = 1
for root, path, files_name in os.walk(nes_files_dir):
    #print(files_name)
    for nes in files_name:
        ret = nes_summary(root + nes)
        ret_list = [idx, '-', '-', '-', nes, ret['num_16k_rom'], ret['num_8k_vrom'], ret['mirroring'], '-']
        ws.append(ret_list)
        idx = idx + 1

wb.save(nes_games_list)
print('Create ' + nes_games_list + 'Ok!')
        

Start create Nes-Games-List-1615875474.825359.xlsx...
Create Nes-Games-List-1615875474.825359.xlsxOk!


In [86]:
## 生成 ROMFILE
## 文件名: Testnes.idx
## 文件格式如下
# Contra2j.nes	0000010		0000000
# Spartanx.nes	0008010		003E000
# Contra1j.nes	0000010		0040000
# F_Reset.bin	0000000		007E000
# Contra_F.nes	0000010		0080000
# ...
import os
import time
from openpyxl import load_workbook,Workbook

def hex2dec(s):
    return int(s, 16)

def dec2hex(s):
    return hex(int(s))

source_file = './Nes-Games-List.xlsx'
idx_file = "./Testnes.idx"

little_file_start_addr = '0000010'
rom_size_sum = 0

f = open(idx_file, 'w')
f.write("F_Reset.bin \t 0000000 \t 007E000 \n")

wb = load_workbook(source_file)
ws1 = wb["Sheet1"]

the_first_item = True

for row in ws1.iter_rows(min_row=1):
    if the_first_item:
        large_file_local_addr = rom_size_sum
        the_first_item = False
    else:
        # large_file_local_addr = '---'
        rom_size = int(row[5].value) * 16 * 1024 + int(row[6].value) * 16 * 1024
        print('Filename: {} , rom_size: {}K'.format(row[4].value, rom_size/1024))
        rom_size_sum += rom_size
      
    f.write("{} \t {} \t {} \n".format(row[4].value, little_file_start_addr, '{:07x}'.format(rom_size_sum).upper() ))
    
f.close()

print('Create ' + idx_file + ' Ok!')



Filename: 1943.nes , rom_size: 128.0K
Filename: 1944.nes , rom_size: 128.0K
Filename: AIsland1.nes , rom_size: 96.0K
Filename: AIsland2.nes , rom_size: 384.0K
Filename: AIsland3.nes , rom_size: 384.0K
Filename: AIsland4.nes , rom_size: 512.0K
Filename: Argos.nes , rom_size: 128.0K
Filename: astyanax.nes , rom_size: 384.0K
Filename: BadDudes.nes , rom_size: 384.0K
Filename: Batman1.nes , rom_size: 384.0K
Filename: Batman2.nes , rom_size: 640.0K
Filename: Batman3.nes , rom_size: 640.0K
Filename: BattKid1.nes , rom_size: 256.0K
Filename: BattKid2.nes , rom_size: 512.0K
Filename: Bayoubil.nes , rom_size: 384.0K
Filename: BuckyOHa.nes , rom_size: 384.0K
Filename: CaptAmer.nes , rom_size: 384.0K
Filename: Castlev1.nes , rom_size: 128.0K
Filename: Castlev2.nes , rom_size: 384.0K
Filename: Chipdal1.nes , rom_size: 384.0K
Filename: Chipdal2.nes , rom_size: 384.0K
Filename: Chipdal3.nes , rom_size: 384.0K
Filename: CodViper.nes , rom_size: 384.0K
Filename: Contra1J.NES , rom_size: 384.0K
Filenam